In [2]:
import math
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression,LinearRegression,BayesianRidge, Lasso
from statistics import mean
from math import sqrt
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM, Bidirectional
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import datetime
import warnings
from tqdm import tqdm
from pathlib import Path
import time
from copy import deepcopy
import os

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [4]:
def read_data():
    calendar = pd.read_csv('calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv('sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    sales_train_validation = pd.read_csv('sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    submission = pd.read_csv('sample_submission.csv')
    return calendar, sell_prices, sales_train_validation, submission

In [5]:
def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 55000000, merge = False):
    
    # melt sales data, get it ready for training
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    sales_train_validation['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'
    
    data = pd.concat([sales_train_validation, test1, test2], axis = 0)
    
    del sales_train_validation, test1, test2
    
    data = data.loc[nrows:]
    
    calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    
    # delete test2 for now, don't delete when we do next stage of testing in June 
    data = data[data['part'] != 'test2']
    
    if merge:
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['d', 'day'], inplace = True, axis = 1)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass

    
    return data

In [6]:
import pandas as pd
pd.options.mode.chained_assignment = None 

calendar, sell_prices, sales_train_validation, submission = read_data()

data = melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 30000000, merge = True)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Melted sales train validation has 58327370 rows and 8 columns
Mem. usage decreased to 3226.27 Mb (9.4% reduction)
Our final dataset to train has 29181090 rows and 18 columns


In [7]:
import datetime 

data["date"] = pd.to_datetime(data["date"])

In [8]:
start_date = '05-01-2015'

mask = (data['date'] > start_date)
data = data.loc[mask]
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
17381460,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,train,2015-05-02,11514,NaN,NaN,NaN,NaN,1,0,1,8.257812
17381461,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2015-05-02,11514,NaN,NaN,NaN,NaN,1,0,1,3.970703
17381462,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,train,2015-05-02,11514,NaN,NaN,NaN,NaN,1,0,1,2.970703
17381463,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,6,train,2015-05-02,11514,NaN,NaN,NaN,NaN,1,0,1,4.640625
17381464,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,3,train,2015-05-02,11514,NaN,NaN,NaN,NaN,1,0,1,2.880859


In [9]:
TARGET = 'sales'      
END_TEST = 1913      
ID_COLS = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

In [10]:
#Don't run this cell unless we need visualization of WRMSSE

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
import gc

from sklearn import preprocessing
import lightgbm as lgb

from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm

class WRMSSEEvaluator_dashboard(object):

    def __init__(self, train_df: pd.DataFrame, valid_df: pd.DataFrame, 
                 calendar: pd.DataFrame, prices: pd.DataFrame):
        train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
        train_target_columns = train_y.columns.tolist()
        weight_columns = train_y.iloc[:, -28:].columns.tolist()

        train_df['all_id'] = 'all'  # for lv1 aggregation

        id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')]\
                     .columns.tolist()
        valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')]\
                               .columns.tolist()

        if not all([c in valid_df.columns for c in id_columns]):
            valid_df = pd.concat([train_df[id_columns], valid_df], 
                                 axis=1, sort=False)

        self.train_df = train_df
        self.valid_df = valid_df
        self.calendar = calendar
        self.prices = prices

        self.weight_columns = weight_columns
        self.id_columns = id_columns
        self.valid_target_columns = valid_target_columns

        weight_df = self.get_weight_df()

        self.group_ids = (
            'all_id',
            'state_id',
            'store_id',
            'cat_id',
            'dept_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            'item_id',
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
        )

        for i, group_id in enumerate(tqdm(self.group_ids)):
            train_y = train_df.groupby(group_id)[train_target_columns].sum()
            scale = []
            for _, row in train_y.iterrows():
                series = row.values[np.argmax(row.values != 0):]
                scale.append(((series[1:] - series[:-1]) ** 2).mean())
            setattr(self, f'lv{i + 1}_scale', np.array(scale))
            setattr(self, f'lv{i + 1}_train_df', train_y)
            setattr(self, f'lv{i + 1}_valid_df', valid_df.groupby(group_id)\
                    [valid_target_columns].sum())

            lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)
            setattr(self, f'lv{i + 1}_weight', lv_weight / lv_weight.sum())

    def get_weight_df(self) -> pd.DataFrame:
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns]\
                    .set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index()\
                   .rename(columns={'level_2': 'd', 0: 'value'})
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)

        weight_df = weight_df.merge(self.prices, how='left',
                                    on=['item_id', 'store_id', 'wm_yr_wk'])
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd'])\
                    .unstack(level=2)['value']\
                    .loc[zip(self.train_df.item_id, self.train_df.store_id), :]\
                    .reset_index(drop=True)
        weight_df = pd.concat([self.train_df[self.id_columns],
                               weight_df], axis=1, sort=False)
        return weight_df

    def rmsse(self, valid_preds: pd.DataFrame, lv: int) -> pd.Series:
        valid_y = getattr(self, f'lv{lv}_valid_df')
        score = ((valid_y - valid_preds) ** 2).mean(axis=1)
        scale = getattr(self, f'lv{lv}_scale')
        return (score / scale).map(np.sqrt) 

    def score(self, valid_preds: Union[pd.DataFrame, 
                                       np.ndarray]) -> float:
        assert self.valid_df[self.valid_target_columns].shape \
               == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, 
                                       columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], 
                                 valid_preds], axis=1, sort=False)

        all_scores = []
        for i, group_id in enumerate(self.group_ids):

            valid_preds_grp = valid_preds.groupby(group_id)[self.valid_target_columns].sum()
            setattr(self, f'lv{i + 1}_valid_preds', valid_preds_grp)
            
            lv_scores = self.rmsse(valid_preds_grp, i + 1)
            setattr(self, f'lv{i + 1}_scores', lv_scores)
            
            weight = getattr(self, f'lv{i + 1}_weight')
            lv_scores = pd.concat([weight, lv_scores], axis=1, 
                                  sort=False).prod(axis=1)
            
            all_scores.append(lv_scores.sum())
            
        self.all_scores = all_scores

        return np.mean(all_scores)
    

    
def create_viz_df(df,lv):
    
    df = df.T.reset_index()
    if lv in [6,7,8,9,11,12]:
        df.columns = [i[0] + '_' + i[1] if i != ('index','') \
                      else i[0] for i in df.columns]
    df = df.merge(calendar.loc[:, ['d','date']], how='left', 
                  left_on='index', right_on='d')
    df['date'] = pd.to_datetime(df.date)
    df = df.set_index('date')
    df = df.drop(['index', 'd'], axis=1)
    
    return df

def create_dashboard(evaluator, by_level_only=False, model_name=None):
    
    wrmsses = [np.mean(evaluator.all_scores)] + evaluator.all_scores
    labels = ['Overall'] + [f'Level {i}' for i in range(1, 13)]

    ## WRMSSE by Level
    plt.figure(figsize=(12,5))
    ax = sns.barplot(x=labels, y=wrmsses)
    ax.set(xlabel='', ylabel='WRMSSE')
    
    #######################ALTERATION##########################
    title = 'WRMSSE by Level'
    if model_name: 
        title = f'WRMSSE by Level for {model_name}'
    plt.title(title, fontsize=20, fontweight='bold')
    #######################ALTERATION-COMPLETE##########################

  
    for index, val in enumerate(wrmsses):
        ax.text(index*1, val+.01, round(val,4), color='black', 
                ha="center")
        
    #######################ALTERATION##########################
    if by_level_only:       # stops function early for quick plotting of 
        plt.show()          # for quick plotting of levels
        return
    #######################ALTERATION-COMPLETE##########################

    # configuration array for the charts
    n_rows = [1, 1, 4, 1, 3, 3, 3, 3, 3, 3, 3, 3]
    n_cols = [1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
    width = [7, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
    height = [4, 3, 12, 3, 9, 9, 9, 9, 9, 9, 9, 9]
    
    for i in range(1,13):
        
        scores = getattr(evaluator, f'lv{i}_scores')
        weights = getattr(evaluator, f'lv{i}_weight')
        
        if i > 1 and i < 9:
            if i < 7:
                fig, axs = plt.subplots(1, 2, figsize=(12, 3))
            else:
                fig, axs = plt.subplots(2, 1, figsize=(12, 8))
                
            ## RMSSE plot
            scores.plot.bar(width=.8, ax=axs[0], color='g')
            axs[0].set_title(f"RMSSE", size=14)
            axs[0].set(xlabel='', ylabel='RMSSE')
            if i >= 4:
                axs[0].tick_params(labelsize=8)
            for index, val in enumerate(scores):
                axs[0].text(index*1, val+.01, round(val,4), color='black', 
                            ha="center", fontsize=10 if i == 2 else 8)
            
            ## Weight plot
            weights.plot.bar(width=.8, ax=axs[1])
            axs[1].set_title(f"Weight", size=14)
            axs[1].set(xlabel='', ylabel='Weight')
            if i >= 4:
                axs[1].tick_params(labelsize=8)
            for index, val in enumerate(weights):
                axs[1].text(index*1, val+.01, round(val,2), color='black', 
                            ha="center", fontsize=10 if i == 2 else 8)
                    
            fig.suptitle(f'Level {i}: {evaluator.group_ids[i-1]}', size=24 ,
                         y=1.1, fontweight='bold')
            plt.tight_layout()
            plt.show()

        trn = create_viz_df(getattr(evaluator, f'lv{i}_train_df')\
                            .iloc[:, -28*3:], i)
        val = create_viz_df(getattr(evaluator, f'lv{i}_valid_df'), i)
        pred = create_viz_df(getattr(evaluator, f'lv{i}_valid_preds'), i)

        n_cate = trn.shape[1] if i < 7 else 9

        fig, axs = plt.subplots(n_rows[i-1], n_cols[i-1], 
                                figsize=(width[i-1],height[i-1]))
        if i > 1:
            axs = axs.flatten()

        ## Time series plot
        for k in range(0, n_cate):

            ax = axs[k] if i > 1 else axs

            trn.iloc[:, k].plot(ax=ax, label='train')
            val.iloc[:, k].plot(ax=ax, label='valid')
            pred.iloc[:, k].plot(ax=ax, label='pred')
            ax.set_title(f"{trn.columns[k]}  RMSSE:{scores[k]:.4f}", size=14)
            ax.set(xlabel='', ylabel='sales')
            ax.tick_params(labelsize=8)
            ax.legend(loc='upper left', prop={'size': 10})

        if i == 1 or i >= 9:
            fig.suptitle(f'Level {i}: {evaluator.group_ids[i-1]}', size=24 , 
                         y=1.1, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
train_df = pd.read_csv('sales_train_validation.csv')
calendar_df = pd.read_csv('calendar.csv')
sell_prices_df = pd.read_csv('sell_prices.csv')
train_df = train_df.loc[:, :'d_' + str(END_TEST)]

train_fold_df = train_df.iloc[:, :-28]
valid_fold_df = train_fold_df.iloc[:, -28:].copy()
# Instantiate an evaluator for scoring validation periodstarting day 1886
e = WRMSSEEvaluator_dashboard(train_fold_df, valid_fold_df, calendar_df, sell_prices_df)


In [10]:
del calendar, sell_prices

In [11]:
import gc 

gc.collect()

82

In [12]:
del sales_train_validation, submission
gc.collect()

calendar, sell_prices, sales_train_validation, submission = read_data()

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns


In [13]:
cols = ["d_{}".format(i) for i in range(1914-28, 1914)]
lastdata = sales_train_validation[["id", 'store_id', 'item_id'] + cols]

# To long form:
lastdata = lastdata.melt(id_vars=["id", 'store_id', 'item_id'], 
                 var_name="d", value_name="sale")

# Add week of year column from 'calendar':
lastdata = pd.merge(lastdata, calendar, how = 'left', 
                left_on = ['d'], right_on = ['d'])

lastdata = lastdata[["id", 'store_id', 'item_id', "sale", "d", "wm_yr_wk"]]

# Add weekly price from 'sell_prices':
lastdata = lastdata.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
lastdata.drop(columns = ['wm_yr_wk'], inplace=True)

# Calculate daily sales in USD:
lastdata['sale_usd'] = lastdata['sale'] * lastdata['sell_price']
lastdata.head()





,id,store_id,item_id,sale,d,sell_price,sale_usd
0,HOBBIES_1_001_CA_1_validation,CA_1,HOBBIES_1_001,1,d_1886,8.257812,8.257812
1,HOBBIES_1_002_CA_1_validation,CA_1,HOBBIES_1_002,1,d_1886,3.970703,3.970703
2,HOBBIES_1_003_CA_1_validation,CA_1,HOBBIES_1_003,0,d_1886,2.970703,0.000000
3,HOBBIES_1_004_CA_1_validation,CA_1,HOBBIES_1_004,0,d_1886,4.640625,0.000000
4,HOBBIES_1_005_CA_1_validation,CA_1,HOBBIES_1_005,1,d_1886,2.880859,2.880859


In [14]:
sales = sales_train_validation

In [15]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import gc

In [16]:
dummies_list = [sales.state_id, sales.store_id, 
                sales.cat_id, sales.dept_id, 
                sales.state_id +'_'+ sales.cat_id, sales.state_id +'_'+ sales.dept_id,
                sales.store_id +'_'+ sales.cat_id, sales.store_id +'_'+ sales.dept_id, 
                sales.item_id, sales.state_id +'_'+ sales.item_id, sales.id]


## First element Level_0 aggregation 'all_sales':
dummies_df_list =[pd.DataFrame(np.ones(sales.shape[0]).astype(np.int8), 
                               index=sales.index, columns=['all']).T]

# List of dummy dataframes:
for i, cats in enumerate(dummies_list):
    dummies_df_list +=[pd.get_dummies(cats, drop_first=False, dtype=np.int8).T]
    
# Concat dummy dataframes in one go:
## Level is constructed for free.
roll_mat_df = pd.concat(dummies_df_list, keys=list(range(12)), 
                        names=['level','id'])#.astype(np.int8, copy=False)

# Save values as sparse matrix & save index for future reference:
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
roll_mat_csr.shape

(42840, 30490)

In [17]:
roll_mat_df.to_pickle('roll_mat_df.pkl')
del dummies_df_list, roll_mat_df, sales_train_validation, calendar, sell_prices
gc.collect()

20

In [18]:
def get_s(drop_days=0):
    # Rollup sales:
    d_name = ['d_' + str(i+1) for i in range(1913-drop_days)]
    sales_train_val = roll_mat_csr * sales[d_name].values

    no_sales = np.cumsum(sales_train_val, axis=1) == 0
    sales_train_val = np.where(no_sales, np.nan, sales_train_val)

    # Denominator of RMSSE / RMSSE
    weight1 = np.nanmean(np.diff(sales_train_val,axis=1)**2,axis=1)
    
    return weight1

In [19]:
S = get_s(drop_days=0)
S.shape

(42840,)

In [20]:
def get_w(sale_usd):
    """
    """
    # Calculate the total sales in USD for each item id:
    total_sales_usd = sale_usd.groupby(
        ['id'], sort=False)['sale_usd'].apply(np.sum).values
    
    # Roll up total sales by ids to higher levels:
    weight2 = roll_mat_csr * total_sales_usd
    
    return 12*weight2/np.sum(weight2)

In [21]:
W = get_w(lastdata[['id','sale_usd']])
W.shape

(42840,)

In [22]:
W_df = pd.DataFrame(W,index = roll_index,columns=['w'])

W_original_df = pd.read_csv('weights_validation.csv')

# Set new index, calculate difference between original and predicted:
W_original_df = W_original_df.set_index(W_df.index)
W_original_df['Predicted'] = W_df.w
W_original_df['diff'] = W_original_df.Weight - W_original_df.Predicted

# See where we are off by more than e-6
m = W_original_df.Weight.values - W_df.w.values > 0.000001
W_original_df[m]

Level_id  Agg_Level_1  Agg_Level_2    Weight  Predicted  \
level id                                                                       
1     CA               Level2           CA            X  0.442371   0.442370   
3     HOBBIES          Level4      HOBBIES            X  0.128079   0.128075   
      HOUSEHOLD        Level4    HOUSEHOLD            X  0.303335   0.303330   
4     FOODS_1          Level5      FOODS_1            X  0.062625   0.062623   
      FOODS_2          Level5      FOODS_2            X  0.154642   0.154639   
      HOBBIES_1        Level5    HOBBIES_1            X  0.122088   0.122084   
      HOUSEHOLD_1      Level5  HOUSEHOLD_1            X  0.229594   0.229592   
      HOUSEHOLD_2      Level5  HOUSEHOLD_2            X  0.073741   0.073738   
5     CA_HOBBIES       Level6           CA      HOBBIES  0.058855   0.058852   
      CA_HOUSEHOLD     Level6           CA    HOUSEHOLD  0.142772   0.142769   
      TX_HOUSEHOLD     Level6           TX    HOUSEHOLD  0.086420   0.086419   
      WI_HOBBIES       Level6           WI      HOBBIES  0.027931   0.027930   
6     CA_FOODS_2       Level7           CA      FOODS_2  0.057655   0.057654   
      CA_HOBBIES_1     Level7           CA    HOBBIES_1  0.056463   0.056460   
      CA_HOUSEHOLD_1   Level7           CA  HOUSEHOLD_1  0.104863   0.104862   
      CA_HOUSEHOLD_2   Level7           CA  HOUSEHOLD_2  0.037909   0.037907   
      TX_FOODS_1       Level7           TX      FOODS_1  0.016016   0.016015   
      WI_FOODS_2       Level7           WI      FOODS_2  0.062561   0.062560   
      WI_HOBBIES_1     Level7           WI    HOBBIES_1  0.026375   0.026374   
7     CA_2_HOUSEHOLD   Level8         CA_2    HOUSEHOLD  0.037630   0.037629   
      CA_3_HOUSEHOLD   Level8         CA_3    HOUSEHOLD  0.055870   0.055869   
8     WI_2_FOODS_2     Level9         WI_2      FOODS_2  0.030535   0.030534   
9     FOODS_2_029     Level10  FOODS_2_029            X  0.002862   0.002861   
      FOODS_2_183     Level10  FOODS_2_183            X  0.002679   0.002677   
      FOODS_3_586     Level10  FOODS_3_586            X  0.005074   0.005073   

                          diff  
level id                        
1     CA              0.000002  
3     HOBBIES         0.000004  
      HOUSEHOLD       0.000005  
4     FOODS_1         0.000002  
      FOODS_2         0.000004  
      HOBBIES_1       0.000004  
      HOUSEHOLD_1     0.000002  
      HOUSEHOLD_2     0.000003  
5     CA_HOBBIES      0.000003  
      CA_HOUSEHOLD    0.000004  
      TX_HOUSEHOLD    0.000001  
      WI_HOBBIES      0.000001  
6     CA_FOODS_2      0.000001  
      CA_HOBBIES_1    0.000003  
      CA_HOUSEHOLD_1  0.000002  
      CA_HOUSEHOLD_2  0.000002  
      TX_FOODS_1      0.000001  
      WI_FOODS_2      0.000002  
      WI_HOBBIES_1    0.000001  
7     CA_2_HOUSEHOLD  0.000001  
      CA_3_HOUSEHOLD  0.000001  
8     WI_2_FOODS_2    0.000001  
9     FOODS_2_029     0.000001  
      FOODS_2_183     0.000001  
      FOODS_3_586     0.000001

In [23]:
SW = W/np.sqrt(S)
sw_df = pd.DataFrame(np.stack((S, W, SW), axis=-1),index = roll_index,columns=['s','w','sw'])
sw_df.to_pickle('sw_df.pkl')

del W_original_df
gc.collect()

53

In [24]:
def rollup(v):
    return roll_mat_csr*v #(v.T*roll_mat_csr.T).T

In [25]:
sw_df = pd.read_pickle('sw_df.pkl')
S = sw_df.s.values
W = sw_df.w.values
SW = sw_df.sw.values

# Load roll up matrix to calcualte aggreagates:
roll_mat_df = pd.read_pickle('roll_mat_df.pkl')
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
del roll_mat_df

In [26]:
del lastdata, sw_df, W_df

gc.collect()

60

In [27]:
def wrmsse_eval(preds, y_true, s = S, w = W, sw=SW):
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(rollup(preds.values-y_true.values))
                            ,axis=1)) * sw)/12 

    return "WRMSSE", score, False

In [28]:
from sklearn.preprocessing import LabelEncoder

def transform(data):
    
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
        
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
        
    data['date'] = pd.to_datetime(data['date'])
    time_features = ['year', 'month', 'quarter', 'week', 'day', 'dayofweek', 'dayofyear']
    dtype = np.int16
    for time_feature in time_features:
        data[time_feature] = getattr(data['date'].dt, time_feature).astype(dtype)
        
    data = reduce_mem_usage(data)
    
    return data

In [40]:
#add in event proximity at some point, see if it gives good results in conjunction with other features

from datetime import date, datetime


def new_features(data):
    
    data_fe = data[['id', 'demand']]
    
    window = 28
    periods = [1, 2, 3, 4, 5, 6, 7]
    group = data_fe.groupby('id')['demand']
    
    # most recent lag data
    for period in periods:
        data_fe['demand_rolling_mean_t' + str(period)] = group.transform(lambda x: x.shift(window).rolling(period).mean())
        data_fe['demand_rolling_std_t' + str(period)] = group.transform(lambda x: x.shift(window).rolling(period).std())
        data_fe['sales_lag_t' + str(period)] = group.transform(lambda x: x.shift(window + period))
    
    
    def get_season(date):
        year = str(date.year)
        seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
                   'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
                   'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}
        if date in seasons['spring']:
            return 0
        if date in seasons['summer']:
            return 1
        if date in seasons['autumn']:
            return 2
        else:
            return 3
        
    date_fe['season'] = data['date'].map(get_season)
    
    data_fe['price_max'] = data.groupby('id')['sell_price'].transform('max')
    data_fe['price_min'] = data.groupby('id')['sell_price'].transform('min')
    data_fe['price_std'] = data.groupby('id')['sell_price'].transform('std')
    data_fe['price_mean'] = data.groupby('id')['sell_price'].transform('mean')  
    
    data_fe['price_norm'] = data_fe['sell_price']/data_fe['price_max']
    data_fe['price_nunique'] = data.groupby('id')['sell_price'].transform('nunique')
    
    data_fe['price_momentum'] = data['sell_price']/data.groupby('id')['sell_price'].transform(lambda x: x.shift(1))
    data_fe['price_momentum_m'] = data['sell_price']/data.groupby('id', 'month')['sell_price'].transform('mean')
    data_fe['price_momentum_y'] = data['sell_price']/data.groupby('id', 'year')['sell_price'].transform('mean')
    
    data_fe['sale_momentum_m'] = data['demand']/data.groupby('id','month')['demand'].transform('mean')
    data_fe['sale_momentum_y'] = data['demand']/data.groupby('id','year')['demand'].transform('mean')
    
    temp = data_fe

    temp.index = pd.to_datetime(data('date'))
    weighted_periods = [1, 2, 3, 4, 5, 6, 7]
    weighted_group = data_fe.groupby('id')['demand']
    
    for weight in weight_periods:
        data_fe['weighted_sales_t' + str(weight)] = weighted_group.ewm(span = weight + window).mean()
    
    data_fe = reduce_mem_usage(data_fe)
        
    lag_rolling_features = [col for col in data_fe.columns if col not in ['id', 'demand']]
    data = pd.concat([data, data_fe[lag_rolling_features]], axis = 1)
    
    del data_fe, temp

    return data

In [30]:
data.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
29181085,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.980469
29181086,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.480469
29181087,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,3.980469
29181088,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.280273
29181089,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.000000


In [31]:

#wdf = pd.read_csv("weight_scale_1886.csv")

In [32]:
START_VAL = '05-22-2015'

In [33]:
#tr_w = tr_x[ID_COLS]

In [68]:

wdf['scaled_weight'] = wdf.weight/np.sqrt(wdf.scale)
wdf = wdf[[ 'Level_id', 'Agg_Level_1', 'Agg_Level_2','scaled_weight']]


######################### level 1 #######################################
tr_w['level_1_sw'] = wdf.loc[wdf['Level_id'] == 'Level1', 'scaled_weight'][0]

######################### level 2 #######################################
level = wdf[wdf.Level_id == 'Level2']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'state_id', 'Agg_Level_2', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['state_id']], level[['state_id', 'scaled_weight']],
                               on='state_id', how='left')[['scaled_weight']]
tr_w['level_2_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 3 #######################################
level = wdf[wdf.Level_id == 'Level3']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'store_id', 'Agg_Level_2', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['store_id']], level[['store_id', 'scaled_weight']],
                               on='store_id', how='left')[['scaled_weight']]
tr_w['level_3_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 4 #######################################
level = wdf[wdf.Level_id == 'Level4']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'cat_id', 'Agg_Level_2', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['cat_id']], level[['cat_id', 'scaled_weight']],
                               on='cat_id', how='left')[['scaled_weight']]
tr_w['level_4_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 5 #######################################
level = wdf[wdf.Level_id == 'Level5']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'dept_id', 'Agg_Level_2', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['dept_id']], level[['dept_id', 'scaled_weight']],
                               on='dept_id', how='left')[['scaled_weight']]
tr_w['level_5_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 6 #######################################
level = wdf[wdf.Level_id == 'Level6']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'state_id', 'cat_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['state_id', 'cat_id']], level[['state_id', 'cat_id', 'scaled_weight']],
                               on=['state_id', 'cat_id'], how='left')[['scaled_weight']]
tr_w['level_6_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 7 #######################################
level = wdf[wdf.Level_id == 'Level7']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'state_id', 'dept_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['state_id', 'dept_id']], level[['state_id', 'dept_id', 'scaled_weight']],
                               on=['state_id', 'dept_id'], how='left')[['scaled_weight']]
tr_w['level_7_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 8 #######################################
level = wdf[wdf.Level_id == 'Level8']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'store_id', 'cat_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['store_id', 'cat_id']], level[['store_id', 'cat_id', 'scaled_weight']],
                               on=['store_id', 'cat_id'], how='left')[['scaled_weight']]
tr_w['level_8_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 9 #######################################
level = wdf[wdf.Level_id == 'Level9']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'store_id', 'dept_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['store_id', 'dept_id']], level[['store_id', 'dept_id', 'scaled_weight']],
                               on=['store_id', 'dept_id'], how='left')[['scaled_weight']]
tr_w['level_9_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 10 #######################################
level = wdf[wdf.Level_id == 'Level10']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'item_id', 'dept_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['item_id']], level[['item_id', 'scaled_weight']],
                               on=['item_id'], how='left')[['scaled_weight']]
tr_w['level_10_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 11 #######################################
level = wdf[wdf.Level_id == 'Level11']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'state_id', 'item_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['state_id', 'item_id']], level[['state_id', 'item_id', 'scaled_weight']],
                               on=['state_id', 'item_id'], how='left')[['scaled_weight']]
tr_w['level_11_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 12 #######################################
level = wdf[wdf.Level_id == 'Level12']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'item_id', 'store_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['item_id', 'store_id']], level[['item_id', 'store_id', 'scaled_weight']],
                               on=['item_id', 'store_id'], how='left')[['scaled_weight']]
tr_w['level_12_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight, wdf
gc.collect()

cols = [col for col in tr_w.columns if 'level' in col]
tr_w = tr_w[cols]

In [70]:
tr_x.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
18021750,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2015-05-23,11517,NaN,NaN,NaN,NaN,0,0,0,8.257812
18021751,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2015-05-23,11517,NaN,NaN,NaN,NaN,0,0,0,3.970703
18021752,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,train,2015-05-23,11517,NaN,NaN,NaN,NaN,0,0,0,2.970703
18021753,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,4,train,2015-05-23,11517,NaN,NaN,NaN,NaN,0,0,0,4.640625
18021754,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2015-05-23,11517,NaN,NaN,NaN,NaN,0,0,0,2.880859
18021755,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,1,train,2015-05-23,11517,NaN,NaN,NaN,NaN,0,0,0,1.000000
18021756,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,1,train,2015-05-23,11517,NaN,NaN,NaN,NaN,0,0,0,7.878906
18021757,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,5,train,2015-05-23,11517,NaN,NaN,NaN,NaN,0,0,0,0.479980
18021758,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2015-05-23,11517,NaN,NaN,NaN,NaN,0,0,0,1.769531
18021759,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2015-05-23,11517,NaN,NaN,NaN,NaN,0,0,0,2.970703


In [ ]:
#Don't run this cell, it will crash the kernel 

roll_1 = csr_matrix(pd.get_dummies(tr_x.d, drop_first=False).values)

roll_2 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.state_id.astype('str'),
                                  drop_first=False).values)

roll_3 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.store_id.astype('str'),
                                   drop_first=False).values)

roll_4 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.cat_id.astype('str'),
                                    drop_first=False).values)

roll_5 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.dept_id.astype('str'),
                                    drop_first=False).values)

roll_6 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.state_id.astype('str') + tr_x.cat_id.astype('str') ,
                                    drop_first=False).values)

roll_7 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.state_id.astype('str') + tr_x.dept_id.astype('str'),
                                    drop_first=False).values)

roll_8 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.store_id.astype('str') + tr_x.cat_id.astype('str'),
                                    drop_first=False).values)

roll_9 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.store_id.astype('str') + tr_x.dept_id.astype('str'),
                                    drop_first=False).values)

roll_10 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.item_id.astype('str') + tr_x.dept_id.astype('str'),
                                   drop_first=False).values)

roll_11 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.state_id.astype('str') + tr_x.item_id.astype('str'),
                                    drop_first=False).values)

roll_12 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.item_id.astype('str') + tr_x.store_id.astype('str'),
                                    drop_first=False).values)

In [73]:
roll_1 = csr_matrix(pd.get_dummies(tr_x.date, drop_first=False).values)

roll_2 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.state_id.astype('str'),
                                  drop_first=False).values)

In [ ]:
roll_3 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.store_id.astype('str'),
                                   drop_first=False).values)

roll_4 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.cat_id.astype('str'),
                                    drop_first=False).values)

roll_5 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.dept_id.astype('str'),
                                    drop_first=False).values)

roll_6 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.state_id.astype('str') + tr_x.cat_id.astype('str') ,
                                    drop_first=False).values)

roll_7 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.state_id.astype('str') + tr_x.dept_id.astype('str'),
                                    drop_first=False).values)

roll_8 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.store_id.astype('str') + tr_x.cat_id.astype('str'),
                                    drop_first=False).values)

In [ ]:
roll_9 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.store_id.astype('str') + tr_x.dept_id.astype('str'),
                                    drop_first=False).values)

roll_10 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.item_id.astype('str') + tr_x.dept_id.astype('str'),
                                   drop_first=False).values)

roll_11 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.state_id.astype('str') + tr_x.item_id.astype('str'),
                                    drop_first=False).values)

roll_12 = csr_matrix(pd.get_dummies(tr_x.date.astype('str') + tr_x.item_id.astype('str') + tr_x.store_id.astype('str'),
                                    drop_first=False).values)

In [45]:
def gradient(roll, level_sw, actuals, preds, n, s):
    diff = actuals - preds
    diffsquared = np.square(diff)
    
    diff = np.reshape(diff,(roll.T.shape[1], -1))
    diffsquared = np.reshape(diffsqaured,(roll.T.shape[1], -1))
    
    diff_sum = roll.T * diff
    diffsquared_sum = roll.T * diffsquared
    
    
    
    
    
    
    
    return level_grad

In [44]:
data.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
29181085,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.980469
29181086,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.480469
29181087,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,3.980469
29181088,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.280273
29181089,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.000000


In [34]:
import sklearn.metrics as metrics 
from datetime import datetime, timedelta
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
from sklearn.model_selection import PredefinedSplit

def run_lgb(data, features, cat_features):
    
    # reset_index
    data.reset_index(inplace = True, drop = True)
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-04-24']
    y_train = x_train['demand']
    test = data[data['date'] >= '2016-04-25']

    # define random hyperparammeters
    params = {
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.09,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        'colsample_bytree': 0.85,
        'colsample_bynode': 0.85,
        'min_data_per_leaf': 25,
        'num_leaves': 200,
        'lambda_l1': 0.4,
        'lambda_l2': 0.4,
        'objective': 'tweedie'}
    
    preds = np.zeros(len(test))
    
    train_set = lgb.Dataset(x_train.iloc[trn_idx][features], y_train.iloc[trn_idx], 
                            categorical_feature = cat_features)
    val_set = lgb.Dataset(x_train.iloc[val_idx][features], y_train.iloc[val_idx], 
                          categorical_feature = cat_features)
        
    model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 100, 
                          valid_sets = [train_set, val_set], verbose_eval = 20, 
                          feval = wrmsse_eval)
    model.save_model("LightGBM_TSS.lgb")
    preds += model.predict(test[features])

    #print('-'*50)
    #print('\n')
        
    test = test[['id', 'date', 'demand']]
    test['demand'] = preds
    return test

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [35]:
def submit(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('May25thsub.csv', index = False)


In [37]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
17381460,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,train,2015-05-02,11514,NaN,NaN,NaN,NaN,1,0,1,8.257812
17381461,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2015-05-02,11514,NaN,NaN,NaN,NaN,1,0,1,3.970703
17381462,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,train,2015-05-02,11514,NaN,NaN,NaN,NaN,1,0,1,2.970703
17381463,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,6,train,2015-05-02,11514,NaN,NaN,NaN,NaN,1,0,1,4.640625
17381464,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,3,train,2015-05-02,11514,NaN,NaN,NaN,NaN,1,0,1,2.880859


In [38]:
data = transform(data)

Mem. usage decreased to 675.18 Mb (54.9% reduction)


In [ ]:
data = new_features(data)

In [ ]:
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 
            'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'year', 
                'month', 'quarter', 'week', 'day', 'dayofweek', 'dayofyear', 'demand_rolling_mean_t7', 
            'demand_rolling_mean_t14', 'demand_rolling_mean_t28', 'demand_rolling_mean_t84',
                'demand_rolling_std_t7', 'demand_rolling_std_t14', 'demand_rolling_std_t28', 'demand_rolling_std_t84']
    
cat_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
                    'event_name_2', 'event_type_2']

In [ ]:
data.head()

In [ ]:
data = reduce_mem_usage(data)

In [ ]:
import gc

gc.collect()

In [ ]:
next_test = run_lgb(data, features, cat_features)
submit(next_test, submission)

In [ ]:
next_test.head()

In [ ]:
data.tail()